In [1]:
#Dependencies 
import pandas as pd 
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect
from config import gkey
import requests
import json
import time

In [2]:
data = "http://www.espn.com/nba/attendance/_/year/2019"
nba_attendance = pd.read_html(data)

In [3]:
nba_attendance=nba_attendance[0].drop([0])
nba_attendance.columns=nba_attendance.iloc[0]
nba_attendance=nba_attendance.drop([1])
nba_attendance=nba_attendance.drop(['GMS','AVG','PCT','RK'],axis=1)
nba_attendance.columns = ["team", "total_attendance"]
nba_attendance["team"]=nba_attendance["team"].str.split().str[-1]
nba_attendance=nba_attendance.sort_values("team")
nba_attendance=nba_attendance[nba_attendance["total_attendance"]!="0"]
nba_attendance

,team,total_attendance
2,76ers,838092
8,Blazers,799345
18,Bucks,721692
3,Bulls,823475
9,Cavaliers,793337
12,Celtics,763584
21,Clippers,710327
27,Grizzlies,607564
28,Hawks,628440
6,Heat,805264


In [4]:
url = "https://en.wikipedia.org/wiki/List_of_National_Basketball_Association_arenas"
nba_capacities = pd.read_html(url)

In [5]:
nba_capacities = nba_capacities[0].drop(['Image','Location','Opened','Ref(s)'],axis=1)
nba_capacities["Lat"]=""
nba_capacities["Lng"]=""
nba_capacities

,Arena,Team(s),Capacity,Lat,Lng
0,American Airlines Arena,Miami Heat,19600,,
1,American Airlines Center,Dallas Mavericks,19200,,
2,Amway Center,Orlando Magic,18846,,
3,AT&T Center,San Antonio Spurs,18418,,
4,Bankers Life Fieldhouse,Indiana Pacers,17923,,
5,Barclays Center,Brooklyn Nets,17732,,
6,Capital One Arena,Washington Wizards,20356,,
7,Chase Center,Golden State Warriors,18064,,
8,Chesapeake Energy Arena,Oklahoma City Thunder,18203,,
9,FedExForum,Memphis Grizzlies,17794,,


In [6]:
params={"key":gkey}
for index,row in nba_capacities.iterrows():
    base_url="https://maps.googleapis.com/maps/api/geocode/json"
    arena=row["Arena"]
    params["address"]=arena
    result=requests.get(base_url, params=params)
    result=result.json()
    try:    
        nba_capacities.loc[index, "Lat"] = result["results"][0]["geometry"]["location"]["lat"]
        nba_capacities.loc[index, "Lng"] = result["results"][0]["geometry"]["location"]["lng"]
        time.sleep(1)
    except (IndexError, KeyError):
        print("results not found...skipping")
        time.sleep(1)
print("Complete")

Complete


In [7]:
nba_capacities["team"]=nba_capacities["Team(s)"].str.split().str[-1]
nba_capacities=nba_capacities.drop(['Team(s)'],axis=1)
nba_capacities=nba_capacities.sort_values("team")
nba_capacities


,Arena,Capacity,Lat,Lng,team
29,Wells Fargo Center,20478,39.9012,-75.172,76ers
14,Moda Center,19441,45.5316,-122.667,Blazers
10,Fiserv Forum,17500,43.0451,-87.9174,Bucks
27,United Center,20917,41.8807,-87.6742,Bulls
16,Rocket Mortgage FieldHouse,19432,41.4966,-81.6881,Cavaliers
25,TD Garden,18624,42.3662,-71.0621,Celtics
20,Staples Center,19068,34.043,-118.267,Clippers
9,FedExForum,17794,35.1381,-90.0506,Grizzlies
22,State Farm Arena,18118,33.7573,-84.3963,Hawks
0,American Airlines Arena,19600,25.7814,-80.187,Heat


In [8]:
nba_attendance.head()


,team,total_attendance
2,76ers,838092
8,Blazers,799345
18,Bucks,721692
3,Bulls,823475
9,Cavaliers,793337


In [9]:
nba_capacities.head()

,Arena,Capacity,Lat,Lng,team
29,Wells Fargo Center,20478,39.9012,-75.172,76ers
14,Moda Center,19441,45.5316,-122.667,Blazers
10,Fiserv Forum,17500,43.0451,-87.9174,Bucks
27,United Center,20917,41.8807,-87.6742,Bulls
16,Rocket Mortgage FieldHouse,19432,41.4966,-81.6881,Cavaliers


In [10]:
nba_data=nba_capacities.merge(nba_attendance,on="team")
nba_data

,Arena,Capacity,Lat,Lng,team,total_attendance
0,Wells Fargo Center,20478,39.9012,-75.172,76ers,838092
1,Moda Center,19441,45.5316,-122.667,Blazers,799345
2,Fiserv Forum,17500,43.0451,-87.9174,Bucks,721692
3,United Center,20917,41.8807,-87.6742,Bulls,823475
4,Rocket Mortgage FieldHouse,19432,41.4966,-81.6881,Cavaliers,793337
5,TD Garden,18624,42.3662,-71.0621,Celtics,763584
6,Staples Center,19068,34.043,-118.267,Clippers,710327
7,FedExForum,17794,35.1381,-90.0506,Grizzlies,607564
8,State Farm Arena,18118,33.7573,-84.3963,Hawks,628440
9,American Airlines Arena,19600,25.7814,-80.187,Heat,805264


In [11]:
engine = create_engine("sqlite:///Resources/nba_data.sqlite")
connection = engine.connect()

In [12]:
nba_data.to_sql("nba_data",connection,index=False, if_exists="replace")

In [14]:
connection.close()